In [0]:
import urllib.request
urllib.request.urlretrieve ("https://zenodo.org/record/1203745/files/UrbanSound8K.tar.gz","a.tar.gz")
import tarfile
tar = tarfile.open("a.tar.gz")
tar.extractall()
tar.close()

In [0]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
from tqdm import tqdm

In [0]:
data=pd.read_csv("UrbanSound8K/metadata/UrbanSound8K.csv")

In [128]:
data.head()

,slice_file_name,fsID,start,end,salience,fold,classID,class
0,100032-3-0-0.wav,100032,0.0,0.317551,1,5,3,dog_bark
1,100263-2-0-117.wav,100263,58.5,62.500000,1,5,2,children_playing
2,100263-2-0-121.wav,100263,60.5,64.500000,1,5,2,children_playing
3,100263-2-0-126.wav,100263,63.0,67.000000,1,5,2,children_playing
4,100263-2-0-137.wav,100263,68.5,72.500000,1,5,2,children_playing


In [129]:
data.tail()

,slice_file_name,fsID,start,end,salience,fold,classID,class
8727,99812-1-2-0.wav,99812,159.522205,163.522205,2,7,1,car_horn
8728,99812-1-3-0.wav,99812,181.142431,183.284976,2,7,1,car_horn
8729,99812-1-4-0.wav,99812,242.691902,246.197885,2,7,1,car_horn
8730,99812-1-5-0.wav,99812,253.209850,255.741948,2,7,1,car_horn
8731,99812-1-6-0.wav,99812,332.289233,334.821332,2,7,1,car_horn


In [0]:
data["duration"]=data["start"]-data["end"]

In [0]:
data["fold"].value_counts()

4     990
5     936
3     925
2     888
1     873
7     838
10    837
6     823
9     816
8     806
Name: fold, dtype: int64

In [0]:
from librosa import display
import librosa

In [0]:
x_train=[]
x_test=[]
y_train=[]
y_test=[]

In [0]:
path="UrbanSound8K/audio/fold"
for i in tqdm(range(len(data))):
    fold_no=str(data.iloc[i]["fold"])
    file=data.iloc[i]["slice_file_name"]
    label=data.iloc[i]["classID"]
    filename=path+fold_no+"/"+file
    #print(filename)
    y,sr=librosa.load(filename)
    mfccs = np.mean(librosa.feature.mfcc(y, sr, n_mfcc=40).T,axis=0)
    #print(mfccs.shape,mfccs.max(),mfccs.min())
    if(fold_no!='10'):
      x_train.append(mfccs)
      y_train.append(label)
    else:
      x_test.append(mfccs)
      y_test.append(label)

100%|██████████| 8732/8732 [46:09<00:00,  4.04it/s]


In [0]:
len(x_train)+len(x_test)

8732

In [0]:
len(data)

8732

In [0]:
train=[]
test=[]

In [0]:
x_train=np.array(x_train)

In [0]:
x_test=np.array(x_test)

In [0]:
y_train=np.array(y_train)
y_test=np.array(y_test)

In [0]:
x_train.shape,x_test.shape,y_train.shape,y_test.shape

((7895, 40), (837, 40), (7895,), (837,))

In [0]:
from keras.utils.np_utils import to_categorical
y_train = to_categorical(y_train, num_classes=10)
y_test = to_categorical(y_test, num_classes=10)

In [0]:
from keras import Sequential
from keras.layers import Dense,Dropout,Activation

In [0]:
model=Sequential()

In [0]:
model.add(Dense(units=256,activation='relu',input_dim=40))
model.add(Dropout(0.3))
model.add(Dense(units=128,activation='relu'))
model.add(Dropout(0.3))
model.add(Dense(units=128,activation='relu'))
model.add(Dropout(0.3))
model.add(Dense(units=10,activation='softmax'))

In [0]:
model.compile(optimizer='adam',loss='categorical_crossentropy',metrics=['accuracy'])

In [0]:
model.fit(x_train,y_train,epochs=20,validation_data=(x_test,y_test),batch_size=50)

Train on 7895 samples, validate on 837 samples
Epoch 1/20
7895/7895 [==============================] - 1s 150us/step - loss: 0.4459 - acc: 0.8509 - val_loss: 1.6475 - val_acc: 0.6129
Epoch 2/20
7895/7895 [==============================] - 1s 149us/step - loss: 0.4265 - acc: 0.8570 - val_loss: 1.6166 - val_acc: 0.5986
Epoch 3/20
7895/7895 [==============================] - 1s 158us/step - loss: 0.4141 - acc: 0.8603 - val_loss: 1.6689 - val_acc: 0.6213
Epoch 4/20
7895/7895 [==============================] - 1s 158us/step - loss: 0.4239 - acc: 0.8546 - val_loss: 1.6623 - val_acc: 0.5795
Epoch 5/20
7895/7895 [==============================] - 1s 159us/step - loss: 0.4100 - acc: 0.8592 - val_loss: 1.8147 - val_acc: 0.5568
Epoch 6/20
7895/7895 [==============================] - 1s 158us/step - loss: 0.3954 - acc: 0.8647 - val_loss: 1.6589 - val_acc: 0.5783
Epoch 7/20
7895/7895 [==============================] - 1s 161us/step - loss: 0.3999 - acc: 0.8635 - val_loss: 1.6248 - val_acc: 0.5914
E

In [0]:
train.append(model.evaluate(x_train,y_train)[1])
test.append(model.evaluate(x_test,y_test)[1])
train,test

837/837 [==============================] - 0s 80us/step


([0.9498416719442685, 0.9498416719442685],
 [0.5937873357584257, 0.5937873357584257])